In [169]:
#py -m pip install faltten_json
import pandas as pd
import numpy as np
import ast
import json
 

In [170]:
# Carga de datos
movies = pd.read_csv('movies_dataset.csv',low_memory=False)

In [171]:
# Quitamos las columnas que no se van a utilizar
movies = movies.drop(columns= ['video', 'imdb_id','adult','original_title','vote_count','poster_path', 'homepage'])

# Eliminamos las filas que contienen errores
movies = movies.drop(movies[movies['budget'].str.contains('jpg')].index)

# Eliminamos pelis duplicadas
movies =movies.drop_duplicates(subset=['id'])

# Generamos la categoria none para las peliculas que no tienen un genero especificado
movies['genres'] = movies[['genres']].replace("[]", "[{'id': 0, 'name': 'None'}]")

# Eliminamos los valores nulos de release_date
movies = movies.dropna(subset='release_date')

# Cambiamos los tipos de datos de las columnas
movies = movies.astype({'budget': 'float64','id': 'int','popularity':'float64'}, copy=False, errors='raise')

# Cambiamos los tipos de datos de las columnas
movies['release_date'] = pd.to_datetime(movies['release_date'], format='%Y-%m-%d',yearfirst=True, errors='coerce')

# Creamos la columna 'release_year'
movies['release_year'] = movies['release_date'].dt.year
movies = movies.astype({'release_year': 'float64'}, copy=False, errors='raise')

# Rellenamos los valores nulos de los campos revenue, budget con el número 0
movies['budget'].fillna(0, inplace=True)
movies['revenue'].fillna(0, inplace=True)

# Cambiamos el nombre de la columna id por id_movie
movies = movies.rename({'id': 'id_movie'}, axis=1)


------------------

In [172]:
# Función que permite calcular el return
def ret(df,col1,col2):
    if df[col1] == 0:
        out = 0
    else:    
        out = df[col2] / df[col1]
    return out

# creamos la columna return
movies['return'] = movies.apply(ret,axis=1,args={'budget','revenue'})

----------------

In [173]:
## TABLA LANZAMIENTOS

release_date_table = movies[['id_movie','release_date']].copy()

release_date_table.loc['year'] = movies['release_date'].dt.year

release_date_table['month'] = movies['release_date'].dt.month_name(locale='es_ES.UTF-8').str.lower()

release_date_table['day_num'] = movies['release_date'].dt.day

release_date_table['day_name'] = movies['release_date'].dt.day_name(locale='es_ES.UTF-8').str.lower()
release_date_table['day_name'] = release_date_table['day_name'].replace({'miã©rcoles': 'miercoles', 'sã¡bado': 'sabado'})

#crear el csv 
release_date_table.to_csv("data/release_date.csv",index=False)


# release_date_table

-----

In [174]:
# TABLA COLLECTIONS

# Extrae los datos anidados en la columna "belongs_to_collection"
collections = pd.json_normalize(data=movies['belongs_to_collection'].dropna().apply(eval))

collections = pd.DataFrame(collections)

# Cambiamos el nombre de la columna id por id_collection y le asignamos type =int
collections = collections[['id','name']]
collections = collections.rename({'id':'id_collection'},axis=1)

collections = collections.astype({'id_collection': 'int'}, copy=False, errors='raise')

# # Eliminamos filas duplicadas
collections = collections.drop_duplicates()

# #crear el csv 
collections.to_csv("data/collections.csv",index=False)


In [175]:
# cremaos la columna 'id_collecction', extraemos sus valores de la columna belongs_to_collection y luego eliminamos esta última.

# # Definimos una expresión regular para extraer el id del campo "id" de la columna belongs_to_collection
regex = r"'id':\s*([\d]+)"

# # Utilizamos la función "str.extract" para extraer el id y guardarlo en una nueva columna 'id_collection'
movies['id_collection'] = movies['belongs_to_collection'].str.extract(regex)

# # moverla al principio para comparar
# movies = movies.insert(0, 'id_collection', movies.pop('id_collection'))

# Eliminamos la columna 'belongs_to_collection'
movies.drop(['belongs_to_collection'], axis= 'columns', inplace=True)

# movies.head(2)

----------------

In [176]:
# TABLA GENRES

genres = movies['genres'].apply(ast.literal_eval) # Convertir el string a lista de diccionarios
genres = pd.DataFrame([dict(x) for x in genres.sum()])[['id', 'name']]
genres = genres.drop_duplicates()
genres.to_csv("data/genres.csv",index=False)



### TABLA MOVIES_GENRES
movies_genres = movies[['id_movie','genres']].copy()

# movies_genres['genres'] = movies_genres[['genres']].replace("[]", "[{'id': 0, 'name': 'None'}]")  # ya esta arriba

movies_genres['genres'] = movies_genres['genres'].apply(ast.literal_eval)

# # Aplicamos explode() para crear filas adicionales para cada elemento en la lista genres
movies_genres = movies_genres.explode('genres')

# # Extraer el valor de id_genre de cada elemento en la lista genres
movies_genres['id_genre'] = movies_genres['genres'].apply(lambda x: x['id'])

# Eliminar la columna genres
movies_genres = movies_genres.drop('genres', axis=1)

# Eliminamos la columna production_companies de la tabla movies
movies = movies.drop('genres', axis=1)




In [177]:
### TABLA MOVIES_production_companies
movies_production_companies = movies[['id_movie','production_companies']].copy()

#reemplazamos los valores nulos por "[{'name': 'None', 'id': 0}]"
movies_production_companies['production_companies'] = movies_production_companies[['production_companies']].replace("[]", "[{'name': 'None', 'id': 0}]")

movies_production_companies['production_companies'] = movies_production_companies['production_companies'].apply(ast.literal_eval)

# Aplicamos explode() para crear filas adicionales para cada elemento en la lista production_companies
movies_production_companies = movies_production_companies.explode('production_companies')

# Extraemos el valor de id_production_companies de cada elemento en la lista production_companies
movies_production_companies['id_production_companies'] = movies_production_companies['production_companies'].apply(lambda x: x['id'])

# # Eliminar la columna production_companies
movies_production_companies = movies_production_companies.drop('production_companies', axis=1)

# Creamos el csv
movies_production_companies.to_csv("data/movies_production_companies.csv",index=False)


# TABLA production_companies

production_companies = movies['production_companies'].apply(ast.literal_eval) # Convertir el string a lista de diccionarios
production_companies = pd.DataFrame([dict(x) for x in production_companies.sum()])[['id', 'name']]
production_companies = production_companies.drop_duplicates()
production_companies.to_csv("data/production_companies.csv",index=False)


# Eliminamos la columna production_companies de la tabla movies
movies = movies.drop('production_companies', axis=1)



In [178]:
### TABLA MOVIES_production_countries

movies_production_countries = movies[['id_movie','production_countries']].copy()

# Reemplazamos los valores nulos por "[{'iso_3166_1': 'none', 'name': 'none'}]"
movies_production_countries['production_countries'] = movies_production_countries[['production_countries']].replace("[]", "[{'iso_3166_1': 'none', 'name': 'none'}]")  # ya esta arriba
movies_production_countries['production_countries'] = movies_production_countries['production_countries'].apply(ast.literal_eval)

# Aplicamos explode() para crear filas adicionales para cada elemento en la lista production_countries
movies_production_countries = movies_production_countries.explode('production_countries')

# Extraer el valor de id_production_countries de cada elemento en la lista production_countries
movies_production_countries['id_production_countries'] = movies_production_countries['production_countries'].apply(lambda x: x['iso_3166_1'])

# Eliminar la columna genres
movies_production_countries = movies_production_countries.drop('production_countries', axis=1)

# Creamos el csv
movies_production_countries.to_csv("data/movies_production_countries.csv",index=False)


### TABLA production_countries

production_countries = movies['production_countries'].apply(ast.literal_eval) # Convertir el string a lista de diccionarios
production_countries = pd.DataFrame([dict(x) for x in production_countries.sum()])[['iso_3166_1', 'name']]
production_countries = production_countries.drop_duplicates()

# Creamos el csv
production_countries.to_csv("data/production_countries.csv",index=False)

# Eliminamos la columna production_countries de la tabla movies
movies = movies.drop('production_countries', axis=1)


In [179]:
### TABLA MOVIES_spoken_languages

movies_spoken_languages = movies[['id_movie','spoken_languages']].copy()

# Reemplazamos los valores nulos por "[{'iso_639_1': 'none', 'name': 'none'}]"
movies_spoken_languages['spoken_languages'] = movies_spoken_languages[['spoken_languages']].replace("[]", "[{'iso_639_1': 'none', 'name': 'none'}]")  
movies_spoken_languages['spoken_languages'] = movies_spoken_languages['spoken_languages'].apply(ast.literal_eval)

# Aplicamos explode() para crear filas adicionales para cada elemento en la lista genres
movies_spoken_languages = movies_spoken_languages.explode('spoken_languages')

# Extraer el valor de id_genre de cada elemento en la lista genres
movies_spoken_languages['id_spoken_languages'] = movies_spoken_languages['spoken_languages'].apply(lambda x: x['iso_639_1'])

# Eliminar la columna spoken_languages
movies_spoken_languages = movies_spoken_languages.drop('spoken_languages', axis=1)

# Creamos el csv
movies_spoken_languages.to_csv("data/movies_spoken_languages.csv",index=False)


### TABLA spoken_languages

spoken_languages = movies['spoken_languages'].apply(ast.literal_eval) # Convertir el string a lista de diccionarios
spoken_languages = pd.DataFrame([dict(x) for x in spoken_languages.sum()])[['iso_639_1', 'name']]
spoken_languages = spoken_languages.drop_duplicates()

# Creamos el csv
spoken_languages.to_csv("data/spoken_languages.csv",index=False)


# Eliminamos la columna production_countries de la tabla movies
movies = movies.drop('spoken_languages', axis=1)


In [180]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45346 entries, 0 to 45465
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   budget             45346 non-null  float64       
 1   id_movie           45346 non-null  int32         
 2   original_language  45335 non-null  object        
 3   overview           44405 non-null  object        
 4   popularity         45346 non-null  float64       
 5   release_date       45346 non-null  datetime64[ns]
 6   revenue            45346 non-null  float64       
 7   runtime            45100 non-null  float64       
 8   status             45266 non-null  object        
 9   tagline            20387 non-null  object        
 10  title              45346 non-null  object        
 11  vote_average       45346 non-null  float64       
 12  release_year       45346 non-null  float64       
 13  return             45346 non-null  float64       
 14  id_col

In [181]:
movies.columns

Index(['budget', 'id_movie', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'release_year', 'return', 'id_collection'],
      dtype='object')

In [182]:
movies = movies[['id_movie','id_collection','title','tagline','overview','original_language','runtime','status','release_date','release_year','popularity','vote_average','budget', 'revenue', 'return']]
movies.head(2)
movies.to_csv("data/movies.csv",index=False)
